# OPEN-CV PROJECT: LIS

# Import all dependencies

In [1]:
import cv2
import time
import mediapipe as mp
import numpy as np
import os
from matplotlib import pyplot as plt

#  Holistic model

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    
def draw_landmarks_custom(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(0,0,255),thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(0,0,255),thickness=1, circle_radius=1),
                             )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,110,10),thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121),thickness=1, circle_radius=1),
                             )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(255,0,0),thickness=3, circle_radius=5),
                             mp_drawing.DrawingSpec(color=(255,0,0),thickness=3, circle_radius=5),
                             )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(255,0,0),thickness=3, circle_radius=5),
                             mp_drawing.DrawingSpec(color=(255,0,0),thickness=3, circle_radius=5),
                             )

In [58]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, 
                          min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        black = np.zeros((1080, 1920, 3))
        ret, frame = cap.read()
        #frame = cv2.flip(frame,1)
        
        # make detection
        image, results = mediapipe_detection(frame, holistic)
        points_rh_x = []
        points_rh_y = []
        if results.right_hand_landmarks:
            for i in range(21):
                points_rh_x.append(results.right_hand_landmarks.landmark[i].x)
                points_rh_y.append(results.right_hand_landmarks.landmark[i].y)
            max_x = max(points_rh_x)
            min_x = min(points_rh_x)
            max_y = max(points_rh_y)
            min_y = min(points_rh_y)

            h, w, c = frame.shape
            start_point = (int(min_x*w), int(min_y*h))

            # Ending coordinate, here (220, 220)
            # represents the bottom right corner of rectangle
            end_point = (int(max_x*w), int(max_y*h))

            # Blue color in BGR
            color = (255, 0, 0)

            # Line thickness of 2 px
            thickness = 1

            # Using cv2.rectangle() method
            # Draw a rectangle with blue line borders of thickness of 2 px
            image = cv2.rectangle(frame, start_point, end_point, color, thickness)
            
            image = cv2.line(frame, (int(min_x*w), int(min_y*h)), (int(min_x*w)+ int((int(max_x*w)-int(min_x*w))/5), int(min_y*h)), color, 5)
            image = cv2.line(frame, (int(max_x*w), int(min_y*h)), (int(max_x*w)- int((int(max_x*w)-int(min_x*w))/5), int(min_y*h)), color, 5)
            image = cv2.line(frame, (int(min_x*w), int(max_y*h)), (int(min_x*w)+ int((int(max_x*w)-int(min_x*w))/5), int(max_y*h)), color, 5)
            image = cv2.line(frame, (int(max_x*w), int(max_y*h)), (int(max_x*w)- int((int(max_x*w)-int(min_x*w))/5), int(max_y*h)), color, 5)
            
            image = cv2.line(frame, (int(min_x*w), int(min_y*h)), (int(min_x*w), int(min_y*h)+ int((int(max_y*h)-int(min_y*h))/5)), color, 5)
            image = cv2.line(frame, (int(min_x*w), int(max_y*h)), (int(min_x*w), int(max_y*h)- int((int(max_y*h)-int(min_y*h))/5)), color, 5)
            image = cv2.line(frame, (int(max_x*w), int(min_y*h)), (int(max_x*w), int(min_y*h)+ int((int(max_y*h)-int(min_y*h))/5)), color, 5)
            image = cv2.line(frame, (int(max_x*w), int(max_y*h)), (int(max_x*w), int(max_y*h)- int((int(max_y*h)-int(min_y*h))/5)), color, 5)





        
        draw_landmarks_custom(frame, results)
        
        cv2.imshow('LIS', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

KeyboardInterrupt: 

# Extract point for training and prediction

In [59]:
def points_detection(results):
    rh = np.array([[points.x, points.y, points.z] for points in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    # lh = np.array([[points.x, points.y, points.z] for points in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    # po = np.array([[points.x, points.y, points.z] for points in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(99)
    # return np.concatenate([lh, rh, po])
    return rh

# Setup Folders for dataset

In [60]:
DATA_PATH = os.path.join('LIS_data')

# Set number of output and number of sample for each label

In [121]:
labels = np.array(['a', 'b', 'c']) # put the entire alphabet in the future
no_sequences = 101

# Collect data for training

In [122]:
data = []
#no_sequences = no_sequences+1

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    for label in labels:
        for id in range(no_sequences):
    
            ret, frame = cap.read()

            # make detection
            image, results = mediapipe_detection(frame, holistic)

            draw_landmarks_custom(frame, results)

            if id == 0:
                cv2.putText(frame, 'STARTING COLLECTION', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1 ,(0,255,0), 4, cv2.LINE_AA)
                #cv2.putText(frame, f'Collecting frame {id} for {label}', (15,32), cv2.FONT_HERSHEY_SIMPLEX, 1 ,(0,255,0), 4, cv2.LINE_AA)
                cv2.imshow('LIS', frame)
                cv2.waitKey(1000)
            else:
                data.append(points_detection(results))
                cv2.putText(frame, f'Collecting frame {id} for {label}', (15,32), cv2.FONT_HERSHEY_SIMPLEX, 1 ,(0,255,0), 4, cv2.LINE_AA) 
                cv2.waitKey(100)
                cv2.imshow('LIS', frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Save dataset for training

In [123]:
import pandas as pd
df = pd.DataFrame(np.array(data))
y=[]
for i in labels:
    y = np.concatenate([y, [i] * (no_sequences-1)])
df['y'] = y
pd.DataFrame(df).to_csv('data.csv')
print(df['y'].value_counts())
df

a    100
b    100
c    100
Name: y, dtype: int64


,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,y
0,0.348107,0.762331,1.513924e-07,0.383289,0.726162,-0.017568,0.416940,0.658651,-0.027685,0.436927,...,0.293984,0.535787,-0.046685,0.314174,0.575904,-0.041996,0.318744,0.616403,-0.033627,a
1,0.260318,0.592471,2.711663e-07,0.299940,0.552299,-0.016765,0.331290,0.489124,-0.024033,0.355295,...,0.204349,0.364516,-0.032928,0.195269,0.328356,-0.037381,0.189226,0.290290,-0.040298,a
2,0.231843,0.605535,4.170414e-07,0.271172,0.566475,-0.020217,0.302637,0.508197,-0.028512,0.330520,...,0.171941,0.387143,-0.028485,0.159777,0.356254,-0.034239,0.151011,0.321798,-0.037927,a
3,0.226641,0.607650,4.768788e-07,0.266352,0.572120,-0.024059,0.296532,0.512238,-0.035645,0.322135,...,0.161482,0.385211,-0.027898,0.148890,0.352443,-0.032937,0.139442,0.317408,-0.035849,a
4,0.236175,0.586870,4.475519e-07,0.274260,0.549129,-0.021151,0.305717,0.494487,-0.032243,0.333054,...,0.172720,0.373300,-0.034118,0.159851,0.342033,-0.040299,0.150559,0.307385,-0.044336,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,0.223077,0.692420,3.894357e-07,0.248824,0.649462,-0.023718,0.254819,0.576426,-0.030063,0.228389,...,0.184256,0.509881,-0.030502,0.191375,0.549300,-0.030929,0.197946,0.575726,-0.025005,c
296,0.226644,0.674913,3.865667e-07,0.253633,0.633263,-0.023803,0.261131,0.561764,-0.028514,0.236224,...,0.192624,0.501300,-0.029023,0.199601,0.540242,-0.030297,0.205384,0.566282,-0.024692,c
297,0.232366,0.667414,4.125832e-07,0.259117,0.625021,-0.023452,0.267601,0.553147,-0.029060,0.243854,...,0.198557,0.491803,-0.029049,0.204604,0.529479,-0.030411,0.209484,0.554525,-0.024792,c
298,0.207053,0.680581,4.259139e-07,0.228962,0.628246,-0.028047,0.226262,0.555153,-0.031905,0.196939,...,0.158189,0.506870,-0.020786,0.165650,0.541598,-0.024444,0.174709,0.563394,-0.021212,c


# Model

In [124]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, MinMaxScaler
import datetime
from datetime import datetime

In [125]:
model_name = datetime.now().strftime("%Y-%m-%d_%H:%M:%S")

tensorboard = TensorBoard(
    log_dir="logs/scalars/" + model_name,
    histogram_freq=0,
    write_graph=True,
    write_images=True,
    profile_batch=0)

In [126]:
label_map = {label:num for num, label in enumerate(labels)}
label_map

{'a': 0, 'b': 1, 'c': 2}

In [127]:
# split into input (X) and output (y) variables
X = np.array(df.iloc[:,0:-1])
y = np.array(df.iloc[:,-1])

# define min max scaler
scaler = MinMaxScaler()

# transform data
X = scaler.fit_transform(X)

# define LabelEncoder
label_encoder = LabelEncoder()

# transform data
integer_encoded = label_encoder.fit_transform(y)

# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y = onehot_encoder.fit_transform(integer_encoded)

In [128]:
# define the keras model
model = Sequential()
model.add(Dense(32, input_dim=X.shape[1], activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(labels.shape[0], activation='softmax'))

In [129]:
# compile the keras model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

In [130]:
# fit the keras model on the dataset
model.fit(X, y, epochs=500, batch_size=20, callbacks=[tensorboard], validation_split=0.2)

Train on 240 samples, validate on 60 samples
Epoch 1/500
240/240 [==============================] - 0s 647us/step - loss: 0.9863 - categorical_accuracy: 0.5500 - val_loss: 1.3721 - val_categorical_accuracy: 0.0000e+00
Epoch 2/500
240/240 [==============================] - 0s 132us/step - loss: 0.8398 - categorical_accuracy: 0.6750 - val_loss: 1.2667 - val_categorical_accuracy: 0.0000e+00
Epoch 3/500
240/240 [==============================] - 0s 200us/step - loss: 0.6601 - categorical_accuracy: 0.8333 - val_loss: 1.2606 - val_categorical_accuracy: 0.0000e+00
Epoch 4/500
240/240 [==============================] - 0s 338us/step - loss: 0.4384 - categorical_accuracy: 0.9292 - val_loss: 0.9706 - val_categorical_accuracy: 0.6500
Epoch 5/500
240/240 [==============================] - 0s 177us/step - loss: 0.2260 - categorical_accuracy: 0.9875 - val_loss: 0.3984 - val_categorical_accuracy: 1.0000
Epoch 6/500
240/240 [==============================] - 0s 172us/step - loss: 0.1069 - categorical_

Epoch 49/500
240/240 [==============================] - 0s 186us/step - loss: 0.0021 - categorical_accuracy: 1.0000 - val_loss: 0.0049 - val_categorical_accuracy: 1.0000
Epoch 50/500
240/240 [==============================] - 0s 384us/step - loss: 0.0019 - categorical_accuracy: 1.0000 - val_loss: 0.0039 - val_categorical_accuracy: 1.0000
Epoch 51/500
240/240 [==============================] - 0s 172us/step - loss: 0.0017 - categorical_accuracy: 1.0000 - val_loss: 0.0034 - val_categorical_accuracy: 1.0000
Epoch 52/500
240/240 [==============================] - 0s 169us/step - loss: 0.0026 - categorical_accuracy: 1.0000 - val_loss: 0.0077 - val_categorical_accuracy: 1.0000
Epoch 53/500
240/240 [==============================] - 0s 164us/step - loss: 0.0037 - categorical_accuracy: 1.0000 - val_loss: 0.0027 - val_categorical_accuracy: 1.0000
Epoch 54/500
240/240 [==============================] - 0s 180us/step - loss: 0.0081 - categorical_accuracy: 0.9958 - val_loss: 0.0326 - val_categoric

Epoch 143/500
240/240 [==============================] - 0s 161us/step - loss: 1.7770e-04 - categorical_accuracy: 1.0000 - val_loss: 8.3031e-04 - val_categorical_accuracy: 1.0000
Epoch 144/500
240/240 [==============================] - 0s 171us/step - loss: 1.4589e-04 - categorical_accuracy: 1.0000 - val_loss: 9.6249e-04 - val_categorical_accuracy: 1.0000
Epoch 145/500
240/240 [==============================] - 0s 132us/step - loss: 1.4562e-04 - categorical_accuracy: 1.0000 - val_loss: 8.2579e-04 - val_categorical_accuracy: 1.0000
Epoch 146/500
240/240 [==============================] - 0s 149us/step - loss: 1.5520e-04 - categorical_accuracy: 1.0000 - val_loss: 8.7209e-04 - val_categorical_accuracy: 1.0000
Epoch 147/500
240/240 [==============================] - 0s 171us/step - loss: 1.3121e-04 - categorical_accuracy: 1.0000 - val_loss: 0.0011 - val_categorical_accuracy: 1.0000
Epoch 148/500
240/240 [==============================] - 0s 140us/step - loss: 1.3615e-04 - categorical_accur

Epoch 189/500
240/240 [==============================] - 0s 165us/step - loss: 7.6273e-05 - categorical_accuracy: 1.0000 - val_loss: 7.2170e-04 - val_categorical_accuracy: 1.0000
Epoch 190/500
240/240 [==============================] - 0s 146us/step - loss: 7.3128e-05 - categorical_accuracy: 1.0000 - val_loss: 7.6324e-04 - val_categorical_accuracy: 1.0000
Epoch 191/500
240/240 [==============================] - 0s 208us/step - loss: 7.3042e-05 - categorical_accuracy: 1.0000 - val_loss: 7.1711e-04 - val_categorical_accuracy: 1.0000
Epoch 192/500
240/240 [==============================] - 0s 151us/step - loss: 7.2370e-05 - categorical_accuracy: 1.0000 - val_loss: 8.1444e-04 - val_categorical_accuracy: 1.0000
Epoch 193/500
240/240 [==============================] - 0s 129us/step - loss: 7.5417e-05 - categorical_accuracy: 1.0000 - val_loss: 7.0131e-04 - val_categorical_accuracy: 1.0000
Epoch 194/500
240/240 [==============================] - 0s 184us/step - loss: 6.8761e-05 - categorical_a

Epoch 235/500
240/240 [==============================] - 0s 138us/step - loss: 3.8364e-05 - categorical_accuracy: 1.0000 - val_loss: 6.3668e-04 - val_categorical_accuracy: 1.0000
Epoch 236/500
240/240 [==============================] - 0s 138us/step - loss: 3.8935e-05 - categorical_accuracy: 1.0000 - val_loss: 6.8751e-04 - val_categorical_accuracy: 1.0000
Epoch 237/500
240/240 [==============================] - 0s 177us/step - loss: 3.7773e-05 - categorical_accuracy: 1.0000 - val_loss: 5.9539e-04 - val_categorical_accuracy: 1.0000
Epoch 238/500
240/240 [==============================] - 0s 239us/step - loss: 3.8295e-05 - categorical_accuracy: 1.0000 - val_loss: 6.2390e-04 - val_categorical_accuracy: 1.0000
Epoch 239/500
240/240 [==============================] - 0s 122us/step - loss: 3.7366e-05 - categorical_accuracy: 1.0000 - val_loss: 5.8973e-04 - val_categorical_accuracy: 1.0000
Epoch 240/500
240/240 [==============================] - 0s 125us/step - loss: 3.6413e-05 - categorical_a

Epoch 281/500
240/240 [==============================] - 0s 162us/step - loss: 2.3874e-05 - categorical_accuracy: 1.0000 - val_loss: 5.2380e-04 - val_categorical_accuracy: 1.0000
Epoch 282/500
240/240 [==============================] - 0s 174us/step - loss: 2.2460e-05 - categorical_accuracy: 1.0000 - val_loss: 5.3588e-04 - val_categorical_accuracy: 1.0000
Epoch 283/500
240/240 [==============================] - 0s 175us/step - loss: 2.2501e-05 - categorical_accuracy: 1.0000 - val_loss: 4.6813e-04 - val_categorical_accuracy: 1.0000
Epoch 284/500
240/240 [==============================] - 0s 155us/step - loss: 2.2353e-05 - categorical_accuracy: 1.0000 - val_loss: 5.5841e-04 - val_categorical_accuracy: 1.0000
Epoch 285/500
240/240 [==============================] - 0s 99us/step - loss: 2.2227e-05 - categorical_accuracy: 1.0000 - val_loss: 5.5123e-04 - val_categorical_accuracy: 1.0000
Epoch 286/500
240/240 [==============================] - 0s 197us/step - loss: 2.2288e-05 - categorical_ac

Epoch 327/500
240/240 [==============================] - 0s 128us/step - loss: 1.4555e-05 - categorical_accuracy: 1.0000 - val_loss: 4.6378e-04 - val_categorical_accuracy: 1.0000
Epoch 328/500
240/240 [==============================] - 0s 133us/step - loss: 1.4436e-05 - categorical_accuracy: 1.0000 - val_loss: 4.6711e-04 - val_categorical_accuracy: 1.0000
Epoch 329/500
240/240 [==============================] - 0s 145us/step - loss: 1.4382e-05 - categorical_accuracy: 1.0000 - val_loss: 4.5185e-04 - val_categorical_accuracy: 1.0000
Epoch 330/500
240/240 [==============================] - 0s 115us/step - loss: 1.4414e-05 - categorical_accuracy: 1.0000 - val_loss: 4.2090e-04 - val_categorical_accuracy: 1.0000
Epoch 331/500
240/240 [==============================] - 0s 149us/step - loss: 1.4174e-05 - categorical_accuracy: 1.0000 - val_loss: 4.6840e-04 - val_categorical_accuracy: 1.0000
Epoch 332/500
240/240 [==============================] - 0s 180us/step - loss: 1.3659e-05 - categorical_a

Epoch 373/500
240/240 [==============================] - 0s 145us/step - loss: 9.6515e-06 - categorical_accuracy: 1.0000 - val_loss: 3.6859e-04 - val_categorical_accuracy: 1.0000
Epoch 374/500
240/240 [==============================] - 0s 217us/step - loss: 9.4449e-06 - categorical_accuracy: 1.0000 - val_loss: 3.7307e-04 - val_categorical_accuracy: 1.0000
Epoch 375/500
240/240 [==============================] - 0s 173us/step - loss: 9.1640e-06 - categorical_accuracy: 1.0000 - val_loss: 3.9652e-04 - val_categorical_accuracy: 1.0000
Epoch 376/500
240/240 [==============================] - 0s 164us/step - loss: 9.2239e-06 - categorical_accuracy: 1.0000 - val_loss: 4.1339e-04 - val_categorical_accuracy: 1.0000
Epoch 377/500
240/240 [==============================] - 0s 262us/step - loss: 9.0061e-06 - categorical_accuracy: 1.0000 - val_loss: 4.0033e-04 - val_categorical_accuracy: 1.0000
Epoch 378/500
240/240 [==============================] - 0s 202us/step - loss: 9.1721e-06 - categorical_a

Epoch 419/500
240/240 [==============================] - 0s 164us/step - loss: 6.7858e-06 - categorical_accuracy: 1.0000 - val_loss: 3.2074e-04 - val_categorical_accuracy: 1.0000
Epoch 420/500
240/240 [==============================] - 0s 108us/step - loss: 6.1131e-06 - categorical_accuracy: 1.0000 - val_loss: 3.5089e-04 - val_categorical_accuracy: 1.0000
Epoch 421/500
240/240 [==============================] - 0s 144us/step - loss: 6.3058e-06 - categorical_accuracy: 1.0000 - val_loss: 3.5817e-04 - val_categorical_accuracy: 1.0000
Epoch 422/500
240/240 [==============================] - 0s 144us/step - loss: 6.2617e-06 - categorical_accuracy: 1.0000 - val_loss: 3.3048e-04 - val_categorical_accuracy: 1.0000
Epoch 423/500
240/240 [==============================] - 0s 126us/step - loss: 6.0699e-06 - categorical_accuracy: 1.0000 - val_loss: 3.3761e-04 - val_categorical_accuracy: 1.0000
Epoch 424/500
240/240 [==============================] - 0s 180us/step - loss: 6.0357e-06 - categorical_a

Epoch 465/500
240/240 [==============================] - 0s 140us/step - loss: 4.2241e-06 - categorical_accuracy: 1.0000 - val_loss: 3.1650e-04 - val_categorical_accuracy: 1.0000
Epoch 466/500
240/240 [==============================] - 0s 198us/step - loss: 4.2485e-06 - categorical_accuracy: 1.0000 - val_loss: 3.0427e-04 - val_categorical_accuracy: 1.0000
Epoch 467/500
240/240 [==============================] - 0s 209us/step - loss: 4.3055e-06 - categorical_accuracy: 1.0000 - val_loss: 2.9011e-04 - val_categorical_accuracy: 1.0000
Epoch 468/500
240/240 [==============================] - 0s 130us/step - loss: 4.1764e-06 - categorical_accuracy: 1.0000 - val_loss: 2.9226e-04 - val_categorical_accuracy: 1.0000
Epoch 469/500
240/240 [==============================] - 0s 107us/step - loss: 4.1347e-06 - categorical_accuracy: 1.0000 - val_loss: 3.1078e-04 - val_categorical_accuracy: 1.0000
Epoch 470/500
240/240 [==============================] - 0s 130us/step - loss: 4.1719e-06 - categorical_a

In [131]:
labels[np.argmax(model.predict(X)[233])]

'c'

# Save models

In [116]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_rh.h5")
print("Saved model to disk")

# Evaluate model

In [132]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [133]:
yhat = model.predict(X)
ytrue = np.argmax(y, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [134]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[200,   0],
        [  0, 100]],

       [[200,   0],
        [  0, 100]],

       [[200,   0],
        [  0, 100]]])

In [135]:
accuracy_score(ytrue, yhat)

1.0

# Testing

In [136]:
from keras.models import model_from_json

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model_rh.h5")
print("Loaded model from disk")

# Set parameters for prediction

In [137]:
threshold = 0.4
min_detection_confidence = 0.5
min_tracking_confidence = 0.5

# Real time detection and prediction

In [139]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=min_detection_confidence, 
                          min_tracking_confidence=min_tracking_confidence) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # make detection
        image, results = mediapipe_detection(frame, holistic)
        
        
        # perform prediction with relative probability
        prediction = labels[np.argmax(model.predict(np.array([points_detection(results)])))]
        pred_prob = int(np.max(model.predict(np.array([points_detection(results)])))*100)

        print(prediction)
        
        # add text with prediction
        if pred_prob > int(0.01*100):
            cv2.putText(frame, f'Lettera: {prediction} ({pred_prob}%)', 
                        (120,200), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        2 ,
                        (0,255,0), 
                        4, 
                        cv2.LINE_AA)
        else:
            cv2.putText(frame, f'Lettera: Unknown', 
                        (120,200), 
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        2 ,
                        (0,255,0), 
                        4, 
                        cv2.LINE_AA)

        #draw_landmarks_custom(frame, results)
  
        cv2.imshow('LIS', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv2.destroyAllWindows()

a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a
a


KeyboardInterrupt: 